In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import os

import core.config as cconfig
import dataflow as cdataf
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpickle as hpickle
import helpers.hprint as hprint

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

In [ ]:
config = cconfig.get_config_from_env()

In [ ]:
dag_config = config.pop("dag_config")

In [ ]:
dag_runner = cdataf.PredictionDagRunner(dag_config)

In [ ]:
cdataf.draw(dag_runner.dag)

In [ ]:
if "set_fit_intervals" in config["backtest_config"].to_dict():
    dag_runner.set_fit_intervals(
        **config["backtest_config", "set_fit_intervals", "func_kwargs"].to_dict()
    )
if "set_predict_intervals" in config["backtest_config"].to_dict():
    dag_runner.set_predict_intervals(
        **config[
            "backtest_config", "set_predict_intervals", "func_kwargs"
        ].to_dict()
    )

In [ ]:
fit_result_bundle = dag_runner.fit()

In [ ]:
payload = cconfig.Config.from_dict({"config": config})

In [ ]:
if (
    "run_oos" in config["backtest_config"].to_dict().keys()
    and config["backtest_config"]
):
    result_bundle = dag_runner.predict()
    payload["fit_result_bundle"] = fit_result_bundle.to_config()
else:
    result_bundle = fit_result_bundle

In [ ]:
result_bundle.payload = payload

In [ ]:
try:
    path = os.path.join(
        config["backtest_config", "experiment_result_dir"], "result_bundle.pkl"
    )
    if True:
        hpickle.to_pickle(result_bundle.to_config().to_dict(), path)
except AssertionError:
    _LOG.warning("Unable to serialize results.")